In [ ]:
from src.subpipe.validate import ValidateBusData
import logging, os, sys, json
import pandas as pd

from src.utils.utils import (
    DATA_MONTH_DAY,
    SUBSCRIBER_DATA_PATH_JSON,
    SUBSCRIBER_FOLDER,
    curr_time_micro,
    sub_logger,
    lat_long_filler
)

logging.basicConfig(
        format="",
        filename=f"logs/notebook-{DATA_MONTH_DAY}.log",
        encoding="utf-8",
        filemode="a",
        level=logging.INFO,
)

In [ ]:
df = pd.read_json(os.path.join(SUBSCRIBER_FOLDER, "04-12.json"))
df = df.sort_values(["VEHICLE_ID", "ACT_TIME"], ascending=True)

# Code For Transform

In [ ]:
df.insert(5, 'TIMESTAMP', 0)
df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,TIMESTAMP,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
237139,219000623,219000624,13DEC2022:00:00:00,2905,37,0,22680,-122.843617,45.504060,12,0.8
237138,219000623,219000624,13DEC2022:00:00:00,2905,47,0,22685,-122.843743,45.504095,12,0.8
237137,219000623,219000624,13DEC2022:00:00:00,2905,54,0,22690,-122.843828,45.504118,12,0.8
237136,219000623,219000626,13DEC2022:00:00:00,2905,181,0,22740,-122.843408,45.505170,12,0.8
237135,219000623,219000626,13DEC2022:00:00:00,2905,196,0,22745,-122.843285,45.505283,11,0.9


In [ ]:
date_format = "%d%b%Y:%H:%M:%S"
opd_sec: pd.Timestamp = pd.to_datetime(pd.to_datetime(df['OPD_DATE'], format=date_format), unit='s')
td: pd.Timedelta = pd.to_timedelta(df['ACT_TIME'], unit='sec')

df['TIMESTAMP'] = opd_sec + td

In [ ]:
new_df = df.copy()

In [ ]:
new_df.reset_index(level=0, inplace=True)
new_df.drop(columns=['index'], inplace=True)

In [ ]:
new_df

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,TIMESTAMP,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,219000623,219000624,13DEC2022:00:00:00,2905,37,2022-12-13 06:18:00,22680,-122.843617,45.504060,12,0.8
1,219000623,219000624,13DEC2022:00:00:00,2905,47,2022-12-13 06:18:05,22685,-122.843743,45.504095,12,0.8
2,219000623,219000624,13DEC2022:00:00:00,2905,54,2022-12-13 06:18:10,22690,-122.843828,45.504118,12,0.8
3,219000623,219000626,13DEC2022:00:00:00,2905,181,2022-12-13 06:19:00,22740,-122.843408,45.505170,12,0.8
4,219000623,219000626,13DEC2022:00:00:00,2905,196,2022-12-13 06:19:05,22745,-122.843285,45.505283,11,0.9
...,...,...,...,...,...,...,...,...,...,...,...
295349,218570609,218570626,13DEC2022:00:00:00,4305,190705,2022-12-13 18:15:03,65703,-122.844028,45.504288,12,0.8
295350,218570609,218570626,13DEC2022:00:00:00,4305,190736,2022-12-13 18:15:08,65708,-122.844162,45.504048,12,0.8
295351,218570609,218570626,13DEC2022:00:00:00,4305,190761,2022-12-13 18:15:13,65713,-122.844325,45.503868,12,0.8
295352,218570609,218570626,13DEC2022:00:00:00,4305,190784,2022-12-13 18:15:18,65718,-122.844518,45.503735,12,0.8
